In [2]:
%load_ext autoreload
%autoreload 2
## allow you to display your plot in the notebook
%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

/home/doto/miniconda3/envs/numerai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/doto/miniconda3/envs/numerai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [30]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [32]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def split_vals(dataset,n): return dataset[:n].copy(), dataset[n:].copy()

In [5]:
types = {'id':'int64',
        'item_nbr':'int32',
        'store_nbr':'int8',
        'unit_sales':'float32',
        'onpromotion':'object'}

In [15]:
# %%time
# train = pd.read_csv('data/train.csv', parse_dates=['date'], infer_datetime_format=True)
# test = pd.read_csv('data/test.csv', parse_dates=['date'],infer_datetime_format=True)

CPU times: user 2.42 s, sys: 32 ms, total: 2.45 s
Wall time: 2.45 s


In [16]:
train.onpromotion.fillna(False,inplace = True) # replace null values by True
train.onpromotion = train.onpromotion.map({'False': False, 'True' : True}) # replace string by actual boolean values
train.onpromotion = train.onpromotion.astype(bool)
%time train.to_feather('data/tmp_train')

CPU times: user 1.04 s, sys: 2.02 s, total: 3.05 s
Wall time: 17.1 s


In [17]:
test.onpromotion.fillna(False,inplace = True) # replace null values by True
test.onpromotion = test.onpromotion.map({'False': False, 'True' : True}) # replace string by actual boolean values
test.onpromotion = test.onpromotion.astype(bool)
%time test.to_feather('data/tmp_test')

CPU times: user 30.1 ms, sys: 48.2 ms, total: 78.3 ms
Wall time: 70.5 ms


In [49]:
%time train = pd.read_feather('data/tmp_train')
%time test = pd.read_feather('data/tmp_test')

CPU times: user 1.21 s, sys: 2.23 s, total: 3.44 s
Wall time: 19.7 s
CPU times: user 35.5 ms, sys: 40.8 ms, total: 76.3 ms
Wall time: 823 ms


In [50]:
%time train.describe(include='all')

CPU times: user 9.75 s, sys: 4.63 s, total: 14.4 s
Wall time: 14.4 s


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
count,7.408058e+07,74080574,7.408057e+07,7.408057e+07,7.408057e+07,74080575
unique,NaN,1174,NaN,NaN,NaN,1
top,NaN,2016-03-05 00:00:00,NaN,NaN,NaN,True
freq,NaN,102947,NaN,NaN,NaN,74080575
first,NaN,2013-01-01 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2016-03-22 00:00:00,NaN,NaN,NaN,NaN
mean,3.704029e+07,NaN,2.722621e+01,8.629667e+05,8.818916e+00,NaN
std,2.138522e+07,NaN,1.640486e+01,4.556073e+05,1.993721e+01,NaN
min,0.000000e+00,NaN,1.000000e+00,9.699500e+04,-1.537200e+04,NaN
25%,1.852014e+07,NaN,1.100000e+01,4.643390e+05,2.000000e+00,NaN


In [51]:
%time test.describe(include='all')

CPU times: user 295 ms, sys: 0 ns, total: 295 ms
Wall time: 295 ms


,id,date,store_nbr,item_nbr,onpromotion
count,3.370464e+06,3370464,3.370464e+06,3.370464e+06,3370464
unique,NaN,16,NaN,NaN,1
top,NaN,2017-08-27 00:00:00,NaN,NaN,True
freq,NaN,210654,NaN,NaN,3370464
first,NaN,2017-08-16 00:00:00,NaN,NaN,NaN
last,NaN,2017-08-31 00:00:00,NaN,NaN,NaN
mean,1.271823e+08,NaN,2.750000e+01,1.244798e+06,NaN
std,9.729693e+05,NaN,1.558579e+01,5.898362e+05,NaN
min,1.254970e+08,NaN,1.000000e+00,9.699500e+04,NaN
25%,1.263397e+08,NaN,1.400000e+01,8.053210e+05,NaN


#### apply log + 1 on the unit_sales an consider negative values as 0 ( in accordance to the competition rules)

In [52]:
train.unit_sales = np.log1p(np.clip(train.unit_sales,0,None))

In [53]:
%time add_datepart(train,'date')

CPU times: user 43.9 s, sys: 6.66 s, total: 50.6 s
Wall time: 50.6 s


In [55]:
train.head()

,id,store_nbr,item_nbr,unit_sales,onpromotion,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,25.0,103665.0,2.079442,True,2013.0,1.0,1.0,1.0,1.0,1.0,False,True,False,True,False,True,1356998400
1,1,25.0,105574.0,0.693147,True,2013.0,1.0,1.0,1.0,1.0,1.0,False,True,False,True,False,True,1356998400
2,2,25.0,105575.0,1.098612,True,2013.0,1.0,1.0,1.0,1.0,1.0,False,True,False,True,False,True,1356998400
3,3,25.0,108079.0,0.693147,True,2013.0,1.0,1.0,1.0,1.0,1.0,False,True,False,True,False,True,1356998400
4,4,25.0,108701.0,0.693147,True,2013.0,1.0,1.0,1.0,1.0,1.0,False,True,False,True,False,True,1356998400


In [60]:
trn, y, nas = proc_df(train,'unit_sales', na_dict=nas)

In [39]:
n_valid = len(test)
n_trn = len(train)-n_valid
train , valid = split_vals(train,n_trn)
train.shape, valid.shape , test.shape

((67339647, 18), (3370464, 18), (3370464, 5))

In [ ]:
def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

def _print_score(m):
    res = [rmse(m.predict(df), y), m.score(df, y)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
set_rf_samples(1_000_000)
%time x =np.array(trn, dtype=np.float32)

adding in front of the code %prun ， runs a profile that tells you which line of code behind the scene took the most time to execute

In [ ]:
m = RandomForestRegressor(n_estimator=20,min_samples_leaf=100,n_jobs=-1)
%time m.fit()